The data for this notebook comes from https://www.kaggle.com/grassknoted/asl-alphabet/data

In [2]:
"""
  1. Mount Google Drive (no need to check if already mounted, it does that for you)
  2. 
"""
from google.colab import drive
drive.mount('/content/gdrive')

root_dir = "/content/gdrive/My Drive/CS474 Final Project/Kaggle ASL Alphabet/"
train_dir = "asl_alphabet_train/asl_alphabet_train"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from IPython.core.ultratb import AutoFormattedTB
from torch.utils.data.sampler import SubsetRandomSampler

__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

AssertionError: ignored

In [0]:
class ASLDataset(Dataset):
  def to_one_hot(self, class_index):
    oh = torch.zeros((len(self.dataset_folder)))
    oh[class_index] = 1
    return oh
    
  def __init__(self, root_path, train_path, size=512):
      self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root_path, train_path) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))

  def __getitem__(self,index):
    sample = self.dataset_folder[index]
    return sample[0], self.to_one_hot(sample[1])
  
  def __len__(self):
    return len(self.dataset_folder)

In [0]:
dataset = ASLDataset(root_dir, train_dir)

In [8]:
batch_size = 10
validation_split = 0.2
shuffle_dataset = True
random_seed= 12

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

NameError: ignored

In [7]:
# This is what was talked about in the video for memory management
num_epochs = 6
valid_frequency = 10

train_losses = []
validation_losses = []

def scope():
  try:
    #your code for calling dataset and dataloader
    gc.collect()
    torch.cuda.empty_cache()


    loop = tqdm(
        total=(len(train_loader) * num_epochs) +
          (len(validation_loader) * (num_epochs // valid_frequency))
        , position = 0)

    for epoch in range(num_epochs):
      sum_loss = 0
      count_loss = 0

      for x, y_truth in train_loader:
        x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)
        optimizer.zero_grad()
        y_hat = model(x)
        y_long = y_truth.long()

        loss = objective(y_hat, y_long)
        sum_loss += loss.item()
        count_loss += 1

        loop.set_description("epoch:{}/{}, loss:{:.4f}, accuracy:{:.4f}.".format(epoch,batch, loss.item(), accuracy))
        loop.update(1)
        loss.backward()
        optimizer.step()

      train_losses.append((epoch, sum_loss / count_loss))

      if epoch % valid_frequency == 0:
        sum_loss = 0
        count_loss = 0
        with torch.no_grad():
          for x, y_truth in valid_loader:
            x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)
            y_hat = model(x)
            loss = objective(y_hat, y_truth.long())
            sum_loss += loss.item()
            count_loss += 1       
          validation_losses.append((epoch, sum_loss / count_loss))
            # validation_accuracies.append([total_batch_counter, sum_acc / count])
          
    # Call your model, figure out loss and accuracy
    
  except:
    __ITB__()
    
scope()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'train_loader' is not defined


# SSH Access

If ssh_connect is True, an ssh server will be set up on the hosting server. Ergo power.

In [0]:
ssh_connect = False
if ssh_connect:
  import random, string, getpass

  password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
  alias = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(8))
  ! echo root:$password | chpasswd

  ! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
  ! mkdir -p /var/run/sshd
  ! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config && echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
  ! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc && echo "export LD_LIBRARY_PATH" >> /root/.bashrc
  get_ipython().system_raw('/usr/sbin/sshd -D &')

  print('sshpass -p {} ssh -o "StrictHostKeyChecking no" -J serveo.net root@{}'.format(password, alias))
  ! ssh -o "StrictHostKeyChecking no" -R $alias:22:localhost:22 serveo.net